# Out-of-matrix prediction using synthetic data

In this notebook we illustrate how to do inference using sideinfo for rows, columns and both.  

### Synthetic data generation

In [ ]:
import smurff
import logging
logging.basicConfig(level = logging.INFO)

import numpy as np
import scipy.sparse as sp

def gen_matrix(shape, num_latent, density = 1.0 ):
    """
    Generate a  matrix by multipling two factors.
    Sparsify if asked.
    """
    X = np.random.normal(size=(shape[0],num_latent))
    W = np.random.normal(size=(shape[1],num_latent))
    Y = np.dot(X, W.transpose()) + np.random.normal(size=shape)
    if density < 1.0:
        Y = sp.coo_matrix(Y)
        _, Y = smurff.make_train_test(Y, density)
    return Y, X ,W

### Train the model

In [ ]:
Y, X, W = gen_matrix((20, 30), 2, 0.8)
Ytrain, Ytest = smurff.make_train_test(Y, 0.1)

In [ ]:
### train the model
import os, shutil

shutil.rmtree("syn-out-matrix-pred")
os.makedirs("syn-out-matrix-pred")

# use X and W as sideinfo
session = smurff.MacauSession(
                       Ytrain     = Ytrain,
                       Ytest      = Ytest,
                       side_info  = [X, W],
                       direct     = True,
                       num_latent = 4,
                       burnin     = 40,
                       nsamples   = 100,
                       save_freq  = 1,
                       save_name = "syn-out-matrix-pred.hdf5",
                       verbose    = 1,)
predictions = session.run()

### Make a PredictSession

In [ ]:
# predictor = smurff.PredictSession.fromRootFile("syn-out-matrix-pred/root.ini")
predictor = session.makePredictSession()
print(predictor)

### Out-of-matrix prediction using side-info for the rows

In [ ]:
# predict with a single feature-vector and for one column with PredictSession.predict_one
# return type: smurff.Prediction
p = predictor.predict_one((X[0],0))
print(p)

In [ ]:
# predict with many feature-vectors in a np.ndarray and for all columns with PredictSession.predict
# return type: numpy.ndarray
p = predictor.predict((X,Ellipsis))
print(p)

In [ ]:
# predict with many feature-vectors in a np.ndarray and for a single columns with PredictSession.predict
# return type: numpy.ndarray
p = predictor.predict((X[1:5],10))
print(p)

### Out-of-matrix prediction using side-info for the columns

In [ ]:
# predict with a single feature-vector and for one row with PredictSession.predict_one
p = predictor.predict_one((0, W[0]))
print(p.pred_avg)

In [ ]:
# predict with many feature-vectors in a np.ndarray and for all rows with PredictSession.predict
# return type: numpy.ndarray
p = predictor.predict((Ellipsis, W))
print(p)

In [ ]:
# predict with many feature-vectors in a np.ndarray and for a single row with PredictSession.predict
# return type: numpy.ndarray
p = predictor.predict((10,W[1:5]))
print(p)

### Out-of-matrix prediction using side-info for rows and columns

In [ ]:
# predict with a single feature-vector for the rows, and for the columns
p = predictor.predict_one((X[0], W[0]))
print(p.pred_avg)

In [ ]:
# predict with many feature-vectors in a np.ndarray  PredictSession.predict
# return type: numpy.ndarray
p = predictor.predict((X, W))
print(p)

In [ ]:
# predict with many feature-vectors in a np.ndarray and for a single row with PredictSession.predict
# return type: numpy.ndarray
p = predictor.predict((X[1:4],W[1:5]))
print(p)